In [1]:
# Общеe

import os
import tqdm
import pathlib
import numpy as np
import pandas as pd
import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
DATA_DIR = pathlib.Path("wow/hakaton/best_of_the_best/")
RS = 398476524

In [3]:
train = pd.read_parquet('train.parquet')
train.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1
0,32.910908,17.376350,77.557840,2.929855,gas1,20.487150,57.633085,49.245392,-44.124381,23.400064,...,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743
1,41.263782,22.419445,47.945514,-25.847472,gas2,21.461239,3.474080,49.659980,-13.553188,-13.047593,...,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716
2,25.580283,17.376350,77.654180,2.799411,gas1,20.487150,28.938295,49.245392,-44.124381,20.610679,...,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043
3,33.756900,17.376350,73.049625,2.953982,gas1,20.487150,28.932311,49.245392,-44.124381,18.107963,...,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795
4,4.223732,38.772534,48.015553,-25.843943,gas2,24.635721,12.011581,51.030938,84.244199,-17.735680,...,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082


In [4]:
train.shape

(153417, 27)

In [5]:
CAT = ["feature4"]
TARGETS = ["target0", "target1"]
FTS = train.filter(like="feature").columns.difference(CAT)

In [6]:
train["gas"] = 0
train.loc[train.feature4=="gas2", "gas"] = 1

In [7]:
FTS = FTS.union(["gas"])
FTS = FTS.drop('feature15')

In [8]:
FTS2 = np.concatenate([FTS, ['target0']])

In [9]:
train['target0'].quantile(0.05)

17.322363949953623

будем валидировать данные на критических значениях , у этого подхода есть минус мы лишаем возможность учиться на критических данных , но можем увидеть как модель маштабируется , что приблизит результат к тесту

In [10]:
train_val1 = train[(train['target0'] < train['target0'].quantile(0.05)) | (train['target0'] > train['target0'].quantile(0.95)) | (train['target1'] > train['target1'].quantile(0.95)) | (train['target1'] < train['target1'].quantile(0.05))]
train_tr1 = train[~((train['target0'] < train['target0'].quantile(0.05)) | (train['target0'] > train['target0'].quantile(0.95)) | (train['target1'] > train['target1'].quantile(0.95)) | (train['target1'] < train['target1'].quantile(0.05)))]
X_val1 = train_val1[FTS]
y_val1 = train_val1[TARGETS[0]]
X_tr1 = train_tr1[FTS]
y_tr1 = train_tr1[TARGETS[0]]

In [11]:
X_val2 = train_val1[FTS]
y_val2 = train_val1[TARGETS[1]]
X_tr2 = train_tr1[FTS]
y_tr2 = train_tr1[TARGETS[1]]

## DL модель

In [12]:
# колбэки
earlystop_callb = EarlyStopping(monitor='val_mape',
                                patience=50,
                                )
checkpoint_callb = ModelCheckpoint(filepath='model_targ0.h5',
                                   monitor='val_mape',
                                   save_best_only=True)

reducelr_callb = ReduceLROnPlateau(monitor = 'val_mape', 
                                   factor=0.1,
                                   patience=30,
                                   verbose=1)

checkpoint_callb1 = ModelCheckpoint(filepath='model_targ1.h5',
                                   monitor='val_mape',
                                   save_best_only=True)

### targ0

In [13]:
inp1 = Input(shape=(24,))

x1 = Dense(512, activation='relu')(inp1)
x1 = Dense(256, activation='relu')(x1)
x1 = Dense(64, activation='relu')(x1)
x1 = Dense(1, activation='linear', name='targ0')(x1)

model_one_targ0 = Model(inp1, x1)

model_one_targ0.compile(optimizer=Adam(0.001), loss='mse', metrics=['mape'])

model_one_targ0.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24)]              0         
                                                                 
 dense (Dense)               (None, 512)               12800     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 targ0 (Dense)               (None, 1)                 65        
                                                                 
Total params: 160,641
Trainable params: 160,641
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model_one_targ0.fit(X_tr1, 
                              y_tr1, 
                              epochs=1000, 
                              batch_size=256, 
                              validation_data=(X_val1, y_val1),
                              callbacks=[reducelr_callb,
                                         checkpoint_callb,
                                         earlystop_callb
                                         ]
                              )

Epoch 1/1000
498/498 [==============================] - 2s 4ms/step - loss: 11.9156 - mape: 2.9891 - val_loss: 0.4179 - val_mape: 1.6122 - lr: 0.0010
Epoch 2/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.1825 - mape: 0.8920 - val_loss: 0.2761 - val_mape: 1.2528 - lr: 0.0010
Epoch 3/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.1728 - mape: 0.8737 - val_loss: 0.4140 - val_mape: 1.3506 - lr: 0.0010
Epoch 4/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.2179 - mape: 0.9736 - val_loss: 0.3362 - val_mape: 1.7219 - lr: 0.0010
Epoch 5/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.2008 - mape: 0.9398 - val_loss: 0.2990 - val_mape: 1.1693 - lr: 0.0010
Epoch 6/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.2300 - mape: 0.9884 - val_loss: 0.3358 - val_mape: 1.6259 - lr: 0.0010
Epoch 7/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.2554 - mape: 1.0159 - val_l

In [15]:
model_one_targ0 = load_model('model_targ0.h5')

In [16]:
pred = np.array(model_one_targ0.predict(X_tr1))
pred1 = np.array(model_one_targ0.predict(X_val1))

816/816 [==============================] - 1s 1ms/step


In [17]:
print(f"MAPE (train): {mean_absolute_percentage_error(y_tr1, pred) * 100:.3f} %")
print(f"MAPE (val): {mean_absolute_percentage_error(y_val1, pred1) * 100:.3f} %")

MAPE (train): 0.379 %
MAPE (val): 0.483 %


In [18]:
model_one_targ0.save('model_targ0.h5')

### targ1

In [19]:
inp1 = Input(shape=(24,))

x1 = Dense(512, activation='relu')(inp1)
x1 = Dense(256, activation='relu')(x1)
x1 = Dense(64, activation='relu')(x1)
x1 = Dense(1, activation='linear', name='targ0')(x1)

model_one_targ1 = Model(inp1, x1)

model_one_targ1.compile(optimizer=Adam(0.001), loss='mse', metrics=['mape'])

model_one_targ1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 24)]              0         
                                                                 
 dense_3 (Dense)             (None, 512)               12800     
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 64)                16448     
                                                                 
 targ0 (Dense)               (None, 1)                 65        
                                                                 
Total params: 160,641
Trainable params: 160,641
Non-trainable params: 0
_________________________________________________________________


In [20]:
history = model_one_targ1.fit(X_tr2, 
                              y_tr2, 
                              epochs=1000, 
                              batch_size=256, 
                              validation_data=(X_val2, y_val2),
                              callbacks=[reducelr_callb,
                                         checkpoint_callb1,
                                         earlystop_callb
                                         ]
                              )

Epoch 1/1000
498/498 [==============================] - 2s 4ms/step - loss: 9.0309 - mape: 11.5812 - val_loss: 3.2976 - val_mape: 6.8609 - lr: 0.0010
Epoch 2/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.7244 - mape: 4.8289 - val_loss: 3.9579 - val_mape: 9.2120 - lr: 0.0010
Epoch 3/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.5659 - mape: 4.5810 - val_loss: 6.7067 - val_mape: 20.8293 - lr: 0.0010
Epoch 4/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.4861 - mape: 4.1345 - val_loss: 2.9547 - val_mape: 5.2804 - lr: 0.0010
Epoch 5/1000
498/498 [==============================] - 2s 3ms/step - loss: 0.5176 - mape: 4.0892 - val_loss: 3.1142 - val_mape: 5.4179 - lr: 0.0010
Epoch 6/1000
498/498 [==============================] - 2s 4ms/step - loss: 0.4909 - mape: 3.8660 - val_loss: 4.9815 - val_mape: 8.4184 - lr: 0.0010
Epoch 7/1000
498/498 [==============================] - 2s 4ms/step - loss: 0.4476 - mape: 3.5592 - val_

In [21]:
model_one_targ1 = load_model('model_targ1.h5')

In [22]:
pred3 = np.array(model_one_targ1.predict(X_tr2))
pred4 = np.array(model_one_targ1.predict(X_val2))

816/816 [==============================] - 1s 897us/step


In [23]:
print(f"MAPE (train): {mean_absolute_percentage_error(y_tr2, pred3) * 100:.3f} %")
print(f"MAPE (val): {mean_absolute_percentage_error(y_val2, pred4) * 100:.3f} %")

MAPE (train): 2.025 %
MAPE (val): 3.254 %


In [24]:
model_one_targ1.save('model_targ1.h5')

# общий mape

In [25]:
pred = np.concatenate([model_one_targ0.predict(X_tr1), model_one_targ1.predict(X_tr2)])
pred1 = np.concatenate([model_one_targ0.predict(X_val1), model_one_targ1.predict(X_val2)])

816/816 [==============================] - 1s 881us/step


In [26]:
print(f"общий DL MAPE (train): {mean_absolute_percentage_error(np.concatenate([y_tr1, y_tr2]), pred) * 100:.3f} %")
print(f"общий DL MAPE (val): {mean_absolute_percentage_error(np.concatenate([y_val1, y_val2]), pred1) * 100:.3f} %")

общий DL MAPE (train): 1.202 %
общий DL MAPE (val): 1.868 %


In [27]:
save_model(model_one_targ1, DATA_DIR.joinpath(f"target1-cb-v1.h5"))
save_model(model_one_targ0, DATA_DIR.joinpath(f"target0-cb-v1.h5"))

# Создаём сабмит

In [28]:
%%writefile predict.py
import pathlib
import pandas as pd
import numpy as np
import keras

DATA_DIR = pathlib.Path(".")
MODEL_FILE_0 = pathlib.Path(__file__).parent.joinpath("target0-cb-v1.h5")
MODEL_FILE_1 = pathlib.Path(__file__).parent.joinpath("target1-cb-v1.h5")

def predict(df: pd.DataFrame) -> pd.DataFrame:
    """
    Вычисление предсказаний.

    Параметры:
        df:
          датафрейм, содержащий строки из тестового множества.
          Типы и имена колонок совпадают с типами и именами в ноутбуке, не содержит `np.nan` или `np.inf`.

    Результат:
        Датафрейм предсказаний.
        Должен содержать то же количество строк и в том же порядке, а также колонки `target0` и `target1`.
    """   

    predictions = {}

    df["gas"] = 0
    df.loc[df.feature4=="gas2", "gas"] = 1
    FTS = df.columns.difference(["feature4"])
    FTS = FTS.drop('feature15')
    df0 = df[FTS]
    

    '''
        блок для тарг0
    ''' 
    model_zero = keras.models.load_model(MODEL_FILE_0)
    
    pred1 = model_zero.predict(df0, verbose=0)

    predictions['target0'] = [i[0] for i in pred1]

    '''
        блок тарг1
    '''
    model_one = keras.models.load_model(MODEL_FILE_1)

    pred2 = model_one.predict(df0, verbose=0)

    predictions['target1'] = [i[0] for i in pred2]

    preds_df = pd.DataFrame(predictions, index=df.index)
    return preds_df

Overwriting predict.py
